## Set-up Project

Install beberapa Library

In [1]:
!pip install protobuf==3.20.3


  Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (679 bytes)
Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.2
    Uninstalling protobuf-5.29.2:
      Successfully uninstalled protobuf-5.29.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
opentelemetry-proto 1.29.0 requires protobuf<6.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.


In [2]:
! pip install -q --upgrade langchain langchain-openai langchain-core langchain_community docx2txt pypdf  langchain_chroma sentence_transformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.2 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 5.29.2 which is incompatible.


In [3]:
import langchain
print(langchain.__version__)

0.3.13


### Basic Component in LLM

##### Load OpenAI Key and LangSmith Key

- Mengintegrasikan LLM provider yang akan digunakan selama project berlangsung, dalam hal ini adalah Open AI
- Mengintegrasikan LangSmith sebagai dashboard untuk bisa monitoring setiap pertanyaan yang dikirimkan ke model dan jawaban yang diberikan oleh model

In [6]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE"

In [7]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "YOUR_API_KEY_HERE"
os.environ["LANGCHAIN_PROJECT"] = "chatbot-stunting-test"

##### Test LLM

In [12]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm_response = llm.invoke("Apa itu stunting")

llm_response

AIMessage(content='Stunting adalah kondisi di mana pertumbuhan fisik seorang anak terhambat, biasanya diukur dengan tinggi badan yang lebih rendah dari standar yang ditetapkan untuk usia dan jenis kelamin tertentu. Stunting biasanya disebabkan oleh kekurangan gizi kronis, terutama pada 1.000 hari pertama kehidupan, yaitu sejak kehamilan hingga usia dua tahun. Selain faktor gizi, stunting juga dapat dipengaruhi oleh faktor kesehatan, lingkungan, dan sosial.\n\nAnak yang mengalami stunting tidak hanya memiliki tinggi badan yang rendah, tetapi juga berisiko mengalami masalah kesehatan jangka panjang, seperti gangguan perkembangan kognitif, serta peningkatan risiko penyakit di masa dewasa. Oleh karena itu, stunting menjadi perhatian penting bagi kesehatan masyarakat dan pembangunan sumber daya manusia di berbagai negara. Penanganan stunting memerlukan pendekatan holistik yang melibatkan peningkatan gizi, kesehatan, sanitasi, dan pendidikan bagi orang tua dan masyarakat.', additional_kwargs

#### Parsing Output

In [13]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(llm_response)

'Stunting adalah kondisi di mana pertumbuhan fisik seorang anak terhambat, biasanya diukur dengan tinggi badan yang lebih rendah dari standar yang ditetapkan untuk usia dan jenis kelamin tertentu. Stunting biasanya disebabkan oleh kekurangan gizi kronis, terutama pada 1.000 hari pertama kehidupan, yaitu sejak kehamilan hingga usia dua tahun. Selain faktor gizi, stunting juga dapat dipengaruhi oleh faktor kesehatan, lingkungan, dan sosial.\n\nAnak yang mengalami stunting tidak hanya memiliki tinggi badan yang rendah, tetapi juga berisiko mengalami masalah kesehatan jangka panjang, seperti gangguan perkembangan kognitif, serta peningkatan risiko penyakit di masa dewasa. Oleh karena itu, stunting menjadi perhatian penting bagi kesehatan masyarakat dan pembangunan sumber daya manusia di berbagai negara. Penanganan stunting memerlukan pendekatan holistik yang melibatkan peningkatan gizi, kesehatan, sanitasi, dan pendidikan bagi orang tua dan masyarakat.'

### RETRIEVAL

Menyiapkan dokumen mulai dari:
- loading the document
- spliting the document
- embedding the document
- storing the document into vector db

Dokumen eksternal yang digunakan:
- Buku Panduan Strategi Nasional Percepatan Pencegahan Stunting
- Peraturan Menteri Kesehatan Republik Indonesia Nomor 2 Tahun 2020 tentang
Standar Antropometri Anak.
